In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import functools
%config IPCompleter.greedy=True

In [8]:
train_file_path = tf.keras.utils.get_file("data_stocks.csv", "file:///D:/data_stocks.csv")
print(train_file_path)

C:\Users\Bomao\.keras\datasets\data_stocks.csv


In [3]:
CSV_COLUMNS = ['DATE', 'SP500']

LABEL_COLUMN = 'SP500'
#LABELS = [0]

def get_dataset(file_path):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=12, # 为了示例更容易展示，手动设置较小的值
      label_name=LABEL_COLUMN,
      select_columns = CSV_COLUMNS,
      na_value="?",
      num_epochs=1,
      ignore_errors=True)
  return dataset

raw_train_data = get_dataset(train_file_path)

#x = raw_train_data[:1]
#print(raw_train_data)

date_data, sp500_data = next(iter(raw_train_data)) # 第一个批次
print("DATE: \n", date_data, "\n")
print("SP500: \n", sp500_data)

DATE: 
 OrderedDict([('DATE', <tf.Tensor: shape=(12,), dtype=int32, numpy=
array([1493912940, 1492696140, 1493404020, 1493134200, 1493663160,
       1491587880, 1493042220, 1494001080, 1491232140, 1491594060,
       1492522680, 1492106640])>)]) 

SP500: 
 tf.Tensor(
[2390.34 2342.06 2383.39 2387.33 2393.37 2361.17 2372.12 2392.09 2349.3
 2357.32 2343.58 2336.07], shape=(12,), dtype=float32)


In [4]:
date_sum = 0
sp500_sum = 0
#for i in date_data:
#    print(i)
#    date_sum+=float(i)
    
for j in sp500_data:
    sp500_sum+=j
#date_avg=all/len(date_data)
sp500_avg=sp500_sum/len(sp500_data)
#print("\n",date_avg)
print("\n",sp500_avg)


 tf.Tensor(2367.3452, shape=(), dtype=float32)


In [5]:
def process_continuous_data(mean, data):
  # 标准化数据
    data = tf.cast(data, tf.float32) * 1/(2*mean);
    
    return tf.reshape(data, [-1, 1])

MEANS = { 
    'SP500' : sp500_avg
}

numerical_columns = []

#for feature in MEANS.keys():
num_col = tf.feature_column.numeric_column('SP500', normalizer_fn=functools.partial(process_continuous_data, sp500_avg)) 
numerical_columns.append(num_col)
    

print(numerical_columns)


    


[NumericColumn(key='SP500', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function process_continuous_data at 0x0000025B3C416558>, <tf.Tensor: shape=(), dtype=float32, numpy=2367.3452>))]


In [6]:
preprocessing_layer = tf.keras.layers.DenseFeatures(numerical_columns)

model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

train_data = raw_train_data.shuffle(500)
#test_data = raw_test_data
model.fit(train_data, epochs=20)

Epoch 1/20
Consider rewriting this model with the Functional API.


ValueError: in user code:

    d:\python 3.7\lib\site-packages\tensorflow\python\keras\engine\training.py:805 train_function  *
        return step_function(self, iterator)
    d:\python 3.7\lib\site-packages\tensorflow\python\keras\engine\training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    d:\python 3.7\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    d:\python 3.7\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    d:\python 3.7\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    d:\python 3.7\lib\site-packages\tensorflow\python\keras\engine\training.py:788 run_step  **
        outputs = model.train_step(data)
    d:\python 3.7\lib\site

In [ ]:
#iterator = raw_train_data.as_numpy_iterator()
#print(next(iterator))